## Bajo el listado de todas las series por lenguaje

In [ ]:
import requests
import csv
import threading
from queue import Queue

API_KEY = "API_KEY"  # Reemplaza con tu clave de TMDb
BASE_URL = "https://api.themoviedb.org/3/discover/tv"
NUM_THREADS = 25  # Número de threads en paralelo
REGION = "AR"  # Código de región para Argentina

# Archivo CSV para exportar los datos
csv_filename = "series_tmdb_argentina_idioma.csv"
lock = threading.Lock()

# Lista de idiomas para descargar series (puedes añadir más idiomas si lo deseas)
LANGUAGES = ["es", "en", "fr", "ja"]  # Ejemplo: español, inglés, francés, alemán, italiano, portugués (, "de", "it", "pt", "ru")

# Lista para almacenar los datos temporalmente
series_data = []

def fetch_data(language, page_queue):
    while not page_queue.empty():
        page = page_queue.get()
        params = {
            "api_key": API_KEY,
            "language": "es-ES",  # Este es el idioma para la descripción general, no para el idioma original
            "sort_by": "popularity.desc",
            "watch_region": REGION,  # Filtrar por disponibilidad en Argentina
            "page": page,
            "original_language": language  # Filtro por idioma original
        }
        response = requests.get(BASE_URL, params=params)
        
        if response.status_code == 200:
            results = response.json().get("results", [])
            page_data = [(
                serie.get('id'),
                serie.get('name', 'Desconocido'),
                serie.get('first_air_date', 'Desconocido'),
                serie.get('backdrop_path', 'N/A'),
                serie.get('genre_ids', []),
                serie.get('origin_country', []),
                serie.get('original_language', 'N/A'),
                serie.get('original_name', 'Desconocido'),
                serie.get('overview', 'N/A'),
                serie.get('popularity', 0),
                serie.get('poster_path', 'N/A'),
                serie.get('vote_average', 0),
                serie.get('vote_count', 0)
            ) for serie in results]
            
            with lock:
                series_data.extend(page_data)
        else:
            print(f"⚠️ Error en la página {page} para el idioma {language}: {response.status_code} - {response.text}")
        
        page_queue.task_done()

def process_language(language):
    print(f"🌐 Procesando el idioma: {language}")
    page_queue = Queue()
    
    # Asegúrate de que el parámetro 'original_language' esté presente
    params = {
        "api_key": API_KEY,
        "language": "es-ES",  # Este es el idioma para la descripción general
        "sort_by": "popularity.desc",
        "watch_region": REGION,  # Filtrar por disponibilidad en Argentina
        "page": 1,
        "original_language": language  # Filtro por idioma original
    }
    
    # Solicitar la primera página para obtener el número total de páginas
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        # Aquí se obtiene el total de páginas que corresponde al idioma especificado
        total_pages = response.json().get("total_pages", 1)
        print(f"📢 Total de páginas para el idioma {language}: {total_pages}")
        
        # Poner las páginas en la cola para procesar
        for page in range(1, min(total_pages, 500) + 1):
            page_queue.put(page)
        
        threads = []
        for _ in range(NUM_THREADS):
            thread = threading.Thread(target=fetch_data, args=(language, page_queue))
            thread.start()
            threads.append(thread)
        
        for thread in threads:
            thread.join()
    else:
        print(f"⚠️ Error al obtener datos iniciales para el idioma {language}: {response.status_code} - {response.text}")

# Iterar por todos los idiomas en la lista LANGUAGES
for language in LANGUAGES:
    process_language(language)

# Guardar los datos en un archivo CSV
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["ID", "Nombre", "Año de Inicio", "Backdrop Path", "Géneros", "País de Origen", "Idioma Original", "Nombre Original", "Descripción", "Popularidad", "Poster Path", "Puntaje Promedio", "Cantidad de Votos"])
    writer.writerows(series_data)

print(f"✅ Archivo CSV generado: {csv_filename}")


🌐 Procesando el idioma: es
📢 Total de páginas para el idioma es: 9744
🌐 Procesando el idioma: en
📢 Total de páginas para el idioma en: 9744
🌐 Procesando el idioma: fr
📢 Total de páginas para el idioma fr: 9744
🌐 Procesando el idioma: ja
📢 Total de páginas para el idioma ja: 9744
✅ Archivo CSV generado: series_tmdb_argentina_idioma.csv


## Busco traer la cantidad de temporadas por serie 

In [ ]:
import requests
import csv
import time
import threading
from queue import Queue

API_KEY = "API_KEY"  # Reemplaza con tu clave de TMDb
CSV_INPUT_PATH = "C:/Users/Martin/OneDrive/Maestría/18- Web Minning/series_tmdb_argentina_idioma.csv"
CSV_OUTPUT_PATH = "C:/Users/Martin/OneDrive/Maestría/18- Web Minning/series_tmdb_seasons.csv"
BASE_URL = "https://api.themoviedb.org/3/tv/"
NUM_THREADS = 45  # Número de hilos para procesamiento concurrente
lock = threading.Lock()

def get_season_count(series_id):
    """Consulta la API de TMDb para obtener la cantidad de temporadas de una serie."""
    url = f"{BASE_URL}{series_id}?api_key={API_KEY}&language=es-ES"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data.get("number_of_seasons", "Desconocido")
    else:
        print(f"⚠️ Error al obtener datos de la serie {series_id}: {response.status_code}")
        return "Error"

def worker(queue, results, total_series):
    """Hilo de trabajo para procesar las series en paralelo."""
    while not queue.empty():
        series_id, series_name, index = queue.get()
        season_count = get_season_count(series_id)
        
        with lock:
            results.append([series_id, series_name, season_count])
            print(f"📊 Progreso: {index}/{total_series} ({(index/total_series)*100:.2f}%) - {series_name}")
        
        queue.task_done()
        time.sleep(0.5)  # Esperar para evitar rate limits

def process_csv():
    """Lee el CSV original, obtiene la cantidad de temporadas en paralelo y guarda un nuevo archivo."""
    queue = Queue()
    results = []
    
    with open(CSV_INPUT_PATH, mode='r', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        header = next(reader)  # Leer la cabecera
        
        total_series = sum(1 for _ in reader)  # Contar total de series
        infile.seek(0)  # Reiniciar lectura del archivo
        next(reader)  # Saltar cabecera
        
        for index, row in enumerate(reader, start=1):
            series_id = row[0]  # ID de la serie
            series_name = row[1]  # Nombre de la serie
            queue.put((series_id, series_name, index))
    
    threads = []
    for _ in range(NUM_THREADS):
        thread = threading.Thread(target=worker, args=(queue, results, total_series))
        thread.start()
        threads.append(thread)
    
    queue.join()  # Esperar a que la cola termine
    
    for thread in threads:
        thread.join()
    
    with open(CSV_OUTPUT_PATH, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(["ID", "Nombre", "Temporadas"])
        writer.writerows(results)
    
    print(f"✅ Archivo generado: {CSV_OUTPUT_PATH}")

if __name__ == "__main__":
    process_csv()

📊 Progreso: 37/40000 (0.09%) - Gran hermano (Australia)
📊 Progreso: 30/40000 (0.07%) - Amar es para siempre
📊 Progreso: 5/40000 (0.01%) - Alla mot alla med Filip och Fredrik
📊 Progreso: 21/40000 (0.05%) - Des squelettes dans le placard
📊 Progreso: 14/40000 (0.03%) - The One Show
📊 Progreso: 29/40000 (0.07%) - DC's Legends of Tomorrow
📊 Progreso: 38/40000 (0.10%) - Elif
📊 Progreso: 45/40000 (0.11%) - NDR Talk Show
📊 Progreso: 22/40000 (0.06%) - SOKO München
📊 Progreso: 6/40000 (0.01%) - The Price Is Right
📊 Progreso: 12/40000 (0.03%) - De Wereld Draait Door
📊 Progreso: 13/40000 (0.03%) - WWE Main Event
📊 Progreso: 28/40000 (0.07%) - ちびまる子ちゃん
📊 Progreso: 31/40000 (0.08%) - Mere Ghar Aayi Ek Nanhi Pari
📊 Progreso: 35/40000 (0.09%) - The Walking Dead
📊 Progreso: 34/40000 (0.08%) - This Old House
📊 Progreso: 11/40000 (0.03%) - Les Mystères de l'amour
📊 Progreso: 16/40000 (0.04%) - La casa de David
📊 Progreso: 40/40000 (0.10%) - Kaahin Kissii Roz
📊 Progreso: 7/40000 (0.02%) - De Slimste Mens

## Este de abajo solo sirve para traer el provider segun la serie, no nos sirve para saber tambien la temporada, se baja igual por si el que trae los providers por serie falla

In [ ]:
import requests
import csv
import threading
import time
import random
from queue import Queue

# Configuración
API_KEY = "API_KEY"
BASE_URL = "https://api.themoviedb.org/3/tv/{}/watch/providers"
COUNTRY_CODE = "AR"  # Argentina
NUM_THREADS = 20  # Se redujo a 20 para evitar saturación
CSV_INPUT_PATH = "C:/Users/Martin/OneDrive/Maestría/18- Web Minning/series_tmdb_argentina_idioma.csv"
CSV_OUTPUT_PATH = "C:/Users/Martin/OneDrive/Maestría/18- Web Minning/series_providers.csv"

# Sincronización
lock = threading.Lock()
id_queue = Queue()
series_data = []
total_series = 0
processed_series = 0


def fetch_providers(tv_id):
    url = BASE_URL.format(tv_id)
    params = {"api_key": API_KEY}

    try:
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json().get("results", {}).get(COUNTRY_CODE, {})
            time.sleep(random.uniform(0.2, 0.3))  # Espera aleatoria para no saturar la API
            if data:
                return data
            else:
                return {"message": "No disponible"}

        elif response.status_code == 429:
            print(f"⚠️ Rate limit alcanzado. Esperando 5 segundos para reintentar... (ID: {tv_id})")
            time.sleep(5)
            return fetch_providers(tv_id)  # Reintento simple

        else:
            return {"error": f"Error {response.status_code}: {response.text}"}

    except requests.RequestException as e:
        return {"error": f"Excepción en la solicitud: {str(e)}"}


def worker():
    global processed_series
    while not id_queue.empty():
        tv_id = id_queue.get()
        providers_info = fetch_providers(tv_id)

        with lock:
            series_data.append({"id": tv_id, **providers_info})
            processed_series += 1
            print(f"Progreso: {processed_series}/{total_series} series procesadas")

        id_queue.task_done()


# Leer IDs desde el archivo CSV
with open(CSV_INPUT_PATH, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    id_list = [row["ID"] for row in reader]
    total_series = len(id_list)
    for tv_id in id_list:
        id_queue.put(tv_id)

# Crear y arrancar los threads
threads = []
for _ in range(NUM_THREADS):
    thread = threading.Thread(target=worker)
    thread.start()
    threads.append(thread)

# Esperar a que todos los threads terminen
for thread in threads:
    thread.join()

# Guardar resultados en un nuevo CSV
with open(CSV_OUTPUT_PATH, mode='w', newline='', encoding='utf-8') as file:
    fieldnames = list(series_data[0].keys())  # id ya viene dentro de los datos
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(series_data)

print(f"✅ Archivo CSV generado: {CSV_OUTPUT_PATH}")


## Busco convertir el csv de la cantidad de temporadas por serie a un csv con una fila por temporada

In [3]:
import csv

# Nombre del archivo original y del nuevo
archivo_entrada = 'series_tmdb_seasons.csv'
archivo_salida = 'series_expandido.csv'

# Leer archivo original y expandir por temporada
with open(archivo_entrada, newline='', encoding='utf-8') as f_entrada, \
     open(archivo_salida, 'w', newline='', encoding='utf-8') as f_salida:

    lector = csv.DictReader(f_entrada)
    campos_salida = ['ID', 'Nombre', 'Temporada']
    escritor = csv.DictWriter(f_salida, fieldnames=campos_salida)
    escritor.writeheader()

    for fila in lector:
        id_serie = fila['ID'].strip()
        nombre = fila['Nombre'].strip()
        try:
            total_temporadas = int(fila['Temporadas'])
        except ValueError:
            print(f"⚠️ Temporadas no válido para '{nombre}', se omite.")
            continue

        for temporada in range(1, total_temporadas + 1):
            escritor.writerow({'ID': id_serie, 'Nombre': nombre, 'Temporada': temporada})

print(f"✅ CSV expandido generado como '{archivo_salida}'")


✅ CSV expandido generado como 'series_expandido.csv'


## Ahora que obtuve el ID de la serie y el numero de la temporada, utilizo ese csv para pasar como parametro en la API https://developer.themoviedb.org/reference/tv-season-watch-providers para obtener los providers por temporada de cada serie

In [ ]:
import csv
import requests
import time
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import re

API_KEY = 'API_KEY'
PAIS = 'AR'
ENTRADA_CSV = 'series_expandido.csv'
SALIDA_CSV = 'series_providers_seasons.csv'
CARPETA_PROGRESO = 'progreso_series'
THREADS = 45
INTERVALO = 0.2  # Intervalo entre peticiones para evitar rate limits
BLOQUE_TAMANO = 10000  # Guarda cada 10.000 filas

contador = 0
lock = threading.Lock()
bloque = []

# Asegurarse de que exista la carpeta
os.makedirs(CARPETA_PROGRESO, exist_ok=True)

# Detectar último bloque generado
bloques_existentes = sorted([
    int(re.findall(r'bloque_(\d+)\.csv', f)[0])
    for f in os.listdir(CARPETA_PROGRESO)
    if re.match(r'bloque_\d+\.csv', f)
])

if bloques_existentes:
    bloque_num = max(bloques_existentes) + 1
    filas_procesadas = (bloque_num - 1) * BLOQUE_TAMANO
    print(f"🔄 Reanudando desde fila {filas_procesadas}")
else:
    bloque_num = 1
    filas_procesadas = 0
    print("🚀 Comenzando desde cero")

def obtener_proveedores(tv_id, temporada):
    url = f"https://api.themoviedb.org/3/tv/{tv_id}/season/{temporada}/watch/providers"
    params = {'api_key': API_KEY}
    try:
        response = requests.get(url, params=params)
        data = response.json()
        flatrate = data.get('results', {}).get(PAIS, {}).get('flatrate', [])
        return flatrate
    except Exception as e:
        print(f"❌ Error al consultar ID {tv_id}, temporada {temporada}: {e}")
        return []

def guardar_bloque(bloque_local, num_bloque):
    archivo = os.path.join(CARPETA_PROGRESO, f"bloque_{num_bloque:05d}.csv")
    with open(archivo, 'w', newline='', encoding='utf-8') as f:
        campos = ['ID', 'Nombre', 'Temporada', 'Provider ID', 'Provider Name']
        escritor = csv.DictWriter(f, fieldnames=campos)
        escritor.writeheader()
        escritor.writerows(bloque_local)
    print(f"📁 Guardado bloque {num_bloque} con {len(bloque_local)} filas")

def tarea(fila):
    tv_id = fila['ID'].strip()
    nombre = fila['Nombre'].strip()
    temporada = fila['Temporada'].strip()

    proveedores = obtener_proveedores(tv_id, temporada)
    time.sleep(INTERVALO)

    if not proveedores:
        return [{
            'ID': tv_id,
            'Nombre': nombre,
            'Temporada': temporada,
            'Provider ID': '',
            'Provider Name': ''
        }]
    else:
        return [{
            'ID': tv_id,
            'Nombre': nombre,
            'Temporada': temporada,
            'Provider ID': p.get('provider_id'),
            'Provider Name': p.get('provider_name')
        } for p in proveedores]

# Leer filas del CSV de entrada
with open(ENTRADA_CSV, newline='', encoding='utf-8') as f_entrada:
    lector = csv.DictReader(f_entrada)
    filas = [fila for i, fila in enumerate(lector) if i >= filas_procesadas]
    total = len(filas)
    print(f"📄 Filas a procesar: {total}")

# Procesar con múltiples hilos
with ThreadPoolExecutor(max_workers=THREADS) as executor:
    tareas = [executor.submit(tarea, fila) for fila in filas]
    for future in as_completed(tareas):
        try:
            resultado = future.result()
        except Exception as e:
            print(f"❌ Error en hilo: {e}")
            resultado = []

        with lock:
            bloque.extend(resultado)
            contador += 1
            if contador % 100 == 0 or contador == total:
                print(f"🔄 Procesado (actual): {contador}/{total}")

            if len(bloque) >= BLOQUE_TAMANO:
                guardar_bloque(bloque, bloque_num)
                bloque.clear()
                bloque_num += 1

# Guardar último bloque si quedó algo
if bloque:
    guardar_bloque(bloque, bloque_num)

# Combinar todos los archivos parciales en uno solo final
with open(SALIDA_CSV, 'w', newline='', encoding='utf-8') as f_salida:
    campos = ['ID', 'Nombre', 'Temporada', 'Provider ID', 'Provider Name']
    escritor = csv.DictWriter(f_salida, fieldnames=campos)
    escritor.writeheader()
    for archivo in sorted(os.listdir(CARPETA_PROGRESO)):
        ruta = os.path.join(CARPETA_PROGRESO, archivo)
        with open(ruta, newline='', encoding='utf-8') as f_parcial:
            lector = csv.DictReader(f_parcial)
            escritor.writerows(lector)

print(f"\n✅ Archivo final generado: {SALIDA_CSV}")


🔄 Reanudando desde fila 190000
📄 Filas a procesar: 45140
🔄 Procesado (actual): 100/45140
🔄 Procesado (actual): 200/45140
🔄 Procesado (actual): 300/45140
🔄 Procesado (actual): 400/45140
🔄 Procesado (actual): 500/45140
🔄 Procesado (actual): 600/45140
🔄 Procesado (actual): 700/45140
🔄 Procesado (actual): 800/45140
🔄 Procesado (actual): 900/45140
🔄 Procesado (actual): 1000/45140
🔄 Procesado (actual): 1100/45140
🔄 Procesado (actual): 1200/45140
🔄 Procesado (actual): 1300/45140
🔄 Procesado (actual): 1400/45140
🔄 Procesado (actual): 1500/45140
🔄 Procesado (actual): 1600/45140
🔄 Procesado (actual): 1700/45140
🔄 Procesado (actual): 1800/45140
🔄 Procesado (actual): 1900/45140
🔄 Procesado (actual): 2000/45140
🔄 Procesado (actual): 2100/45140
🔄 Procesado (actual): 2200/45140
🔄 Procesado (actual): 2300/45140
🔄 Procesado (actual): 2400/45140
🔄 Procesado (actual): 2500/45140
🔄 Procesado (actual): 2600/45140
🔄 Procesado (actual): 2700/45140
🔄 Procesado (actual): 2800/45140
🔄 Procesado (actual): 2900/4

## Hago la bajada de las URL de los logos

In [ ]:
import requests
import pandas as pd

# Tu clave API de TMDb (asegúrate de que sea correcta)
API_KEY = 'API_KEY'

# URL de la API de TMDb para obtener proveedores de streaming en un país específico (por ejemplo, Argentina)
url = f'https://api.themoviedb.org/3/watch/providers/movie?api_key={API_KEY}&language=es-AR'

response = requests.get(url)

# Verifica el contenido de la respuesta para entender qué se está recibiendo
data = response.json()

# Imprimir los datos completos para asegurarte de qué contiene la respuesta
print("Datos de la respuesta:", data)

# Extraer los proveedores de la respuesta
providers = data.get('results', [])

# Mostrar los proveedores encontrados
for provider in providers:
    provider_name = provider.get('provider_name')
    logo_path = provider.get('logo_path')
    
    if provider_name and logo_path:
        logo_url = f'https://image.tmdb.org/t/p/w500{logo_path}'
        print(f'Proveedor: {provider_name}, URL del logo: {logo_url}')

# Crear un DataFrame con los nombres de los proveedores y sus URLs de logo
provider_data = []
for provider in providers:
    provider_name = provider.get('provider_name')
    logo_path = provider.get('logo_path')
    
    if provider_name and logo_path:
        logo_url = f'https://image.tmdb.org/t/p/w500{logo_path}'
        provider_data.append([provider_name, logo_url])

# Convertir a DataFrame de pandas para mostrarlo como tabla
df_providers = pd.DataFrame(provider_data, columns=["Provider", "Logo_URL"])
df_providers


# Ruta donde deseas guardar el archivo CSV
save_path = r"C:\Users\Martin\OneDrive\Maestría\18- Web Minning\web-mining\logos\providers_logos.csv"

# Guardar el DataFrame como un archivo CSV
df_providers.to_csv(save_path, index=False)

# Confirmación
print(f"El archivo CSV se ha guardado correctamente en: {save_path}")

Datos de la respuesta: {'results': [{'display_priorities': {'AE': 1, 'AR': 3, 'AT': 4, 'AU': 9, 'BE': 6, 'BG': 2, 'BO': 4, 'BR': 8, 'BY': 0, 'BZ': 0, 'CA': 6, 'CH': 4, 'CL': 3, 'CO': 4, 'CR': 5, 'CY': 0, 'CZ': 4, 'CV': 11, 'DE': 4, 'DK': 5, 'EC': 6, 'EE': 2, 'EG': 2, 'ES': 5, 'FI': 7, 'FR': 4, 'GB': 4, 'GH': 14, 'GR': 2, 'GT': 6, 'HK': 4, 'HN': 5, 'HU': 3, 'ID': 4, 'IE': 4, 'IN': 3, 'IL': 23, 'IT': 4, 'JP': 6, 'LU': 0, 'LT': 2, 'LV': 2, 'MU': 13, 'MX': 2, 'MY': 4, 'MZ': 14, 'NI': 0, 'NL': 9, 'NO': 5, 'NZ': 4, 'PE': 3, 'PL': 2, 'PH': 4, 'PT': 4, 'PY': 5, 'RU': 2, 'SA': 1, 'SG': 6, 'SE': 4, 'SI': 24, 'SK': 4, 'TH': 4, 'TR': 7, 'TW': 6, 'UA': 2, 'UG': 14, 'US': 6, 'VE': 4, 'ZA': 2}, 'display_priority': 2, 'logo_path': '/9ghgSC0MA082EL6HLCW3GalykFD.jpg', 'provider_name': 'Apple TV', 'provider_id': 2}, {'display_priorities': {'AE': 3, 'AO': 0, 'AR': 4, 'AZ': 1, 'AT': 7, 'AU': 12, 'BF': 0, 'BE': 7, 'BO': 19, 'BY': 1, 'BR': 14, 'BZ': 1, 'CA': 8, 'CH': 5, 'CL': 4, 'CO': 3, 'CR': 20, 'CY': 1, '